In [1]:
using Perla1, DataFrames, DataFramesMeta, CSV, Parameters

In [2]:
using Plots
gr(fmt = :png)

Plots.GRBackend()

In [3]:
ϕ_default = 0.9
η_default = [6.5]
ηs = [6.5]
νs = range(0.01, 0.05, length = 10)
δ_Ms = range(0.0225, 0.2, length = 10)
settings = stationary_settings_default()
industry_settings = merge(stationary_settings_default(),
    industry_settings_default(
        ftol = 1e-4, 
        ignore_p_i_deviations = true, 
        trace_iterations = true))

function d_alt(θ_i, μ_i, params)
    @unpack η, ν, ϕ = params
    return (θ_i^ϕ/(ν^(ϕ/2)) + abs(μ_i-1.0)^ϕ/η)^(2/ϕ)
end

d_alt (generic function with 1 method)

# Results by `η`

In [4]:
θs = zeros(0)
μs = zeros(0)
Qs = zeros(0)
Bs = zeros(0)
profit_shares = zeros(0)

for η in ηs
    # fix other parameters other than δ_M
    params = stationary_params_default()
    industry_params = merge(industry_params_default(η = η, d = d_alt, ϕ = ϕ_default), params)
    @time sol = industry_equilibrium(industry_params, industry_settings)
    
    θ = sol.zero[1]
    μ = sol.zero[2]
    @time ss = steadystate(merge(params, (θ = θ, μ = μ)), settings)
    
    append!(θs, θ)
    append!(μs, μ)
    append!(Qs, ss.Q)
    append!(Bs, ss.B)
    append!(profit_shares, ss.profit_share)
end

 19.219225 seconds (63.64 M allocations: 3.131 GiB, 8.50% gc time)
  4.798198 seconds (11.81 M allocations: 608.504 MiB, 9.19% gc time)


In [5]:
results_by_ηs = DataFrame(η = ηs,
    ν = industry_params_default().ν,
    δ_M = stationary_params_default().δ_M,
    θ = θs,
    μ = μs,
    Q = Qs, B = Bs, 
    profit_share = profit_shares)
results_by_ηs.change_by = "eta";
results_by_ηs

,η,ν,δ_M,θ,μ,Q,B,profit_share
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,6.5,0.0178923,0.056,0.0585149,1.55898,0.814186,0.829125,0.170875


# Results by `ν`

In [ ]:
θs = zeros(0)
μs = zeros(0)
Qs = zeros(0)
Bs = zeros(0)
profit_shares = zeros(0)

for ν in νs
    # fix other parameters other than δ_M
    params = stationary_params_default()
    industry_params = merge(industry_params_default(η = η_default, d = d_alt, ϕ = ϕ_default, ν = ν), params)
    @time sol = industry_equilibrium(industry_params, industry_settings)
    
    θ = sol.zero[1]
    μ = sol.zero[2]
    ss = steadystate(merge(params, (θ = θ, μ = μ)), settings)
    
    append!(θs, θ)
    append!(μs, μ)
    append!(Qs, ss.Q)
    append!(Bs, ss.B)
    append!(profit_shares, ss.profit_share)
end

In [ ]:
results_by_νs = DataFrame(η = η_default,
    ν = νs,
    δ_M = stationary_params_default().δ_M,
    θ = θs,
    μ = μs,
    Q = Qs, B = Bs, 
    profit_share = profit_shares)
results_by_νs.change_by = "nu";
results_by_νs

# Results by `δ_M `

In [ ]:
θs = zeros(0)
μs = zeros(0)
Qs = zeros(0)
Bs = zeros(0)
profit_shares = zeros(0)

for δ_M in δ_Ms
    # fix other parameters other than δ_M
    params = stationary_params_default(δ_M = δ_M)
    industry_params = merge(industry_params_default(η = η_default, d = d_alt, ϕ = ϕ_default), params)
    @time sol = industry_equilibrium(industry_params, industry_settings)
    
    θ = sol.zero[1]
    μ = sol.zero[2]
    ss = steadystate(merge(params, (θ = θ, μ = μ)), settings)
    
    append!(θs, θ)
    append!(μs, μ)
    append!(Qs, ss.Q)
    append!(Bs, ss.B)
    append!(profit_shares, ss.profit_share)
end

In [ ]:
results_by_δ_Ms = DataFrame(η = η_default,
    ν = industry_params_default().ν,
    δ_M = δ_Ms,
    θ = θs,
    μ = μs,
    Q = Qs, B = Bs, 
    profit_share = profit_shares)
results_by_δ_Ms.change_by = "delta_M";
results_by_δ_Ms

# Write CSV

In [ ]:
equilibriums = deepcopy(results_by_ηs)
append!(equilibriums, results_by_νs)
append!(equilibriums, results_by_δ_Ms)

# change unicode column names to English
rename!(equilibriums, :η => :eta)
rename!(equilibriums, :ν => :nu)
rename!(equilibriums, :δ_M => :delta_M)
rename!(equilibriums, :μ => :mu)
rename!(equilibriums, :θ => :theta)

CSV.write("industry-equilibrium-variation.csv", equilibriums)
equilibriums